In [1]:
#Import dependencies
import csv
import os
import matplotlib as plt
import pandas as pd
from pathlib import Path
import numpy as np
from importlib import resources


In [2]:
#RR#importing Raw-data 
df = pd.read_csv('Resources\Modern Family Planning_Raw data.csv')
df.head(10)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AFR,Africa,Country,UGA,Uganda,Year,2021,...,NaN,NaN,NaN,NaN,NaN,65.5,NaN,The global indicator represents all women of r...,EN,2023-03-08T05:00:00.000Z
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AFR,Africa,Country,NER,Niger,Year,2020-2021,...,NaN,NaN,NaN,NaN,NaN,36.9,NaN,Niger 2021 PMA Survey Phase 1,EN,2023-03-08T05:00:00.000Z
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AFR,Africa,Country,BFA,Burkina Faso,Year,2020-2021,...,NaN,NaN,NaN,NaN,NaN,58.2,NaN,Burkina Faso 2021 PMA Survey Phase 2,EN,2023-03-08T05:00:00.000Z
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,WPR,Western Pacific,Country,VNM,Viet Nam,Year,2020-2021,...,NaN,NaN,NaN,NaN,NaN,72.1,NaN,The global indicator represents all women of r...,EN,2023-03-08T05:00:00.000Z
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AFR,Africa,Country,CIV,Cote d'Ivoire,Year,2020,...,NaN,NaN,NaN,NaN,NaN,46.1,NaN,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2023-03-08T05:00:00.000Z
5,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,EUR,Europe,Country,MDA,Republic of Moldova,Year,2020,...,NaN,NaN,NaN,NaN,NaN,59.5,NaN,Moldova 2020 Gender and Generations Survey,EN,2023-03-08T05:00:00.000Z
6,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AFR,Africa,Country,ETH,Ethiopia,Year,2020,...,NaN,NaN,NaN,NaN,NaN,62.6,NaN,The global indicator represents all women of r...,EN,2023-03-08T05:00:00.000Z
7,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AFR,Africa,Country,MDG,Madagascar,Year,2020,...,NaN,NaN,NaN,NaN,NaN,65.6,NaN,The global indicator represents all women of r...,EN,2023-03-08T05:00:00.000Z
8,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AMR,Americas,Country,PER,Peru,Year,2020,...,NaN,NaN,NaN,NaN,NaN,65.9,NaN,The global indicator represents all women of r...,EN,2023-03-08T05:00:00.000Z
9,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,text,AFR,Africa,Country,KEN,Kenya,Year,2020,...,NaN,NaN,NaN,NaN,NaN,77.1,NaN,Kenya 2020 PMA Survey Phase 2,EN,2023-03-08T05:00:00.000Z


In [3]:
#RR##checking numbers of Columns
df.columns

Index(['IndicatorCode', 'Indicator', 'ValueType', 'ParentLocationCode',
       'ParentLocation', 'Location type', 'SpatialDimValueCode', 'Location',
       'Period type', 'Period', 'IsLatestYear', 'Dim1 type', 'Dim1',
       'Dim1ValueCode', 'Dim2 type', 'Dim2', 'Dim2ValueCode', 'Dim3 type',
       'Dim3', 'Dim3ValueCode', 'DataSourceDimValueCode', 'DataSource',
       'FactValueNumericPrefix', 'FactValueNumeric', 'FactValueUoM',
       'FactValueNumericLowPrefix', 'FactValueNumericLow',
       'FactValueNumericHighPrefix', 'FactValueNumericHigh', 'Value',
       'FactValueTranslationID', 'FactComments', 'Language', 'DateModified'],
      dtype='object')

In [4]:
#RR#removing unnecessery columns 
drop_colums = df.drop(['ValueType','Location type','Period type','Dim1 type','IsLatestYear','Dim1ValueCode','Dim2 type','Dim2ValueCode',
         'Dim3 type', 'Dim3', 'Dim3ValueCode','DataSourceDimValueCode', 'DataSource','FactValueNumericPrefix','FactValueUoM',
          'FactValueTranslationID', 'FactValueNumericHigh','FactValueNumericHighPrefix','FactValueNumericLow',
         'FactValueNumericLowPrefix','DateModified'], axis=1)
drop_colums.head(10)

,IndicatorCode,Indicator,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,Dim1,Dim2,FactValueNumeric,Value,FactComments,Language
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,2021,Female,15-49 years,65.5,65.5,The global indicator represents all women of r...,EN
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,2020-2021,Female,15-49 years,36.9,36.9,Niger 2021 PMA Survey Phase 1,EN
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,2020-2021,Female,15-49 years,58.2,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,2020-2021,Female,15-49 years,72.1,72.1,The global indicator represents all women of r...,EN
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,2020,Female,15-49 years,46.1,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN
5,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,EUR,Europe,MDA,Republic of Moldova,2020,Female,15-49 years,59.5,59.5,Moldova 2020 Gender and Generations Survey,EN
6,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,ETH,Ethiopia,2020,Female,15-49 years,62.6,62.6,The global indicator represents all women of r...,EN
7,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,MDG,Madagascar,2020,Female,15-49 years,65.6,65.6,The global indicator represents all women of r...,EN
8,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AMR,Americas,PER,Peru,2020,Female,15-49 years,65.9,65.9,The global indicator represents all women of r...,EN
9,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,KEN,Kenya,2020,Female,15-49 years,77.1,77.1,Kenya 2020 PMA Survey Phase 2,EN


In [5]:
#RR#Removing duplicate column
Cleaned_Data = drop_colums.drop(['FactValueNumeric'], axis=1)
Cleaned_Data.head(5)

,IndicatorCode,Indicator,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,Dim1,Dim2,Value,FactComments,Language
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,2021,Female,15-49 years,65.5,The global indicator represents all women of r...,EN
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,2020-2021,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,2020-2021,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,2020-2021,Female,15-49 years,72.1,The global indicator represents all women of r...,EN
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,2020,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN


In [6]:
#RR#Extracted coulmns List
Cleaned_Data.columns

Index(['IndicatorCode', 'Indicator', 'ParentLocationCode', 'ParentLocation',
       'SpatialDimValueCode', 'Location', 'Period', 'Dim1', 'Dim2', 'Value',
       'FactComments', 'Language'],
      dtype='object')

In [7]:
Cleaned_Data.head(10)

,IndicatorCode,Indicator,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,Dim1,Dim2,Value,FactComments,Language
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,2021,Female,15-49 years,65.5,The global indicator represents all women of r...,EN
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,2020-2021,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,2020-2021,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,2020-2021,Female,15-49 years,72.1,The global indicator represents all women of r...,EN
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,2020,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN
5,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,EUR,Europe,MDA,Republic of Moldova,2020,Female,15-49 years,59.5,Moldova 2020 Gender and Generations Survey,EN
6,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,ETH,Ethiopia,2020,Female,15-49 years,62.6,The global indicator represents all women of r...,EN
7,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,MDG,Madagascar,2020,Female,15-49 years,65.6,The global indicator represents all women of r...,EN
8,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AMR,Americas,PER,Peru,2020,Female,15-49 years,65.9,The global indicator represents all women of r...,EN
9,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,KEN,Kenya,2020,Female,15-49 years,77.1,Kenya 2020 PMA Survey Phase 2,EN


In [8]:
#RR#Renaming Colums
Cleaned_Data = Cleaned_Data.rename(columns={'Dim1':'Gender','Dim2':'Age_Group','Value':'Planning_Value_Rate','Language':'Language_Code', 'Location':'Country',
                                            'SpatialDimValueCode':'Country_Code', 'ParentLocation':'ParentLocation_Continent',
                                            'ParentLocationCode':'Patents_Continent_Code'})

In [9]:
#RR#Checking the null values
Cleaned_Data.isnull()

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Period,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
132,False,False,False,False,False,False,False,False,False,False,False,False
133,False,False,False,False,False,False,False,False,False,False,False,False
134,False,False,False,False,False,False,False,False,False,False,False,False
135,False,False,False,False,False,False,False,False,False,False,False,False


In [10]:
#RR#checking duplicates
Cleaned_Data.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
132    False
133    False
134    False
135    False
136    False
Length: 137, dtype: bool

In [11]:
#RR#split up span in to start and end date
Cleaned_Data['Period']

0           2021
1      2020-2021
2      2020-2021
3      2020-2021
4           2020
         ...    
132         2006
133         2005
134    2004-2005
135         2004
136         2001
Name: Period, Length: 137, dtype: object

In [12]:
#RR#Splitting years by pattern '-'
Cleaned_Data['Period'].str.split(pat='-')

0            [2021]
1      [2020, 2021]
2      [2020, 2021]
3      [2020, 2021]
4            [2020]
           ...     
132          [2006]
133          [2005]
134    [2004, 2005]
135          [2004]
136          [2001]
Name: Period, Length: 137, dtype: object

In [13]:
#RR# checking plit where 0=starting year=LHS values
Cleaned_Data['Period'].str.split(pat='-').str[0]

0      2021
1      2020
2      2020
3      2020
4      2020
       ... 
132    2006
133    2005
134    2004
135    2004
136    2001
Name: Period, Length: 137, dtype: object

In [14]:
#RR#adding left hand values to new column name 'Starting_Year'
Cleaned_Data['Starting_Year'] = Cleaned_Data['Period'].str.split(pat='-').str[0]

In [15]:
#RR#checking plit where 1=year_Ending=RHS values
Cleaned_Data['Period'].str.split(pat='-').str[1]

0       NaN
1      2021
2      2021
3      2021
4       NaN
       ... 
132     NaN
133     NaN
134    2005
135     NaN
136     NaN
Name: Period, Length: 137, dtype: object

In [16]:
#RR#adding Right hand values to new column name 'Year_Ending'
Cleaned_Data['Year_Ending'] = Cleaned_Data['Period'].str.split(pat='-').str[1]

In [17]:
Cleaned_Data.head(5)

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Period,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,2021,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,NaN
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,2020-2021,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN,2020,2021
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,2020-2021,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN,2020,2021
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,2020-2021,Female,15-49 years,72.1,The global indicator represents all women of r...,EN,2020,2021
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,2020,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,NaN


In [18]:
#RR# checking null values for column 'year_Ending'
Cleaned_Data.isnull()

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Period,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,False,False,False,False,False,False,False,False,False,False,False,False,False,True
133,False,False,False,False,False,False,False,False,False,False,False,False,False,True
134,False,False,False,False,False,False,False,False,False,False,False,False,False,False
135,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [19]:
#RR#checking records who is carrying null values for year_ending column
Cleaned_Data[Cleaned_Data['Year_Ending'].isna()==1]

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Period,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,2021,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,NaN
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,2020,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,NaN
5,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,EUR,Europe,MDA,Republic of Moldova,2020,Female,15-49 years,59.5,Moldova 2020 Gender and Generations Survey,EN,2020,NaN
6,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,ETH,Ethiopia,2020,Female,15-49 years,62.6,The global indicator represents all women of r...,EN,2020,NaN
7,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,MDG,Madagascar,2020,Female,15-49 years,65.6,The global indicator represents all women of r...,EN,2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,MHL,Marshall Islands,2007,Female,15-49 years,80.5,The global indicator represents all women of r...,EN,2007,NaN
132,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,EUR,Europe,AZE,Azerbaijan,2006,Female,15-49 years,21.5,The global indicator represents all women of r...,EN,2006,NaN
133,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CPV,Cabo Verde,2005,Female,15-49 years,73.2,The global indicator represents all women of r...,EN,2005,NaN
135,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,EUR,Europe,ROU,Romania,2004,Female,15-49 years,46.5,The global indicator represents all women of r...,EN,2004,NaN


In [20]:
#RR#replacing null values by zero
Cleaned_Data['Year_Ending'] = Cleaned_Data['Year_Ending'].fillna(Cleaned_Data['Starting_Year'])
Cleaned_Data.head(10)

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Period,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,2021,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,2021
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,2020-2021,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN,2020,2021
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,2020-2021,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN,2020,2021
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,2020-2021,Female,15-49 years,72.1,The global indicator represents all women of r...,EN,2020,2021
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,2020,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,2020
5,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,EUR,Europe,MDA,Republic of Moldova,2020,Female,15-49 years,59.5,Moldova 2020 Gender and Generations Survey,EN,2020,2020
6,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,ETH,Ethiopia,2020,Female,15-49 years,62.6,The global indicator represents all women of r...,EN,2020,2020
7,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,MDG,Madagascar,2020,Female,15-49 years,65.6,The global indicator represents all women of r...,EN,2020,2020
8,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AMR,Americas,PER,Peru,2020,Female,15-49 years,65.9,The global indicator represents all women of r...,EN,2020,2020
9,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,KEN,Kenya,2020,Female,15-49 years,77.1,Kenya 2020 PMA Survey Phase 2,EN,2020,2020


In [21]:
Cleaned_Data=Cleaned_Data.drop(['Period'],axis=1)
Cleaned_Data.head(5)

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,2021
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN,2020,2021
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN,2020,2021
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,Female,15-49 years,72.1,The global indicator represents all women of r...,EN,2020,2021
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,2020


In [22]:
#RR# Varifying null values for column 'year_Ending'
Cleaned_Data.isnull()

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,False,False,False,False,False,False,False,False,False,False,False,False,False
133,False,False,False,False,False,False,False,False,False,False,False,False,False
134,False,False,False,False,False,False,False,False,False,False,False,False,False
135,False,False,False,False,False,False,False,False,False,False,False,False,False


In [23]:
#RR# checking null values for column 'year_Ending'
Cleaned_Data.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
132    False
133    False
134    False
135    False
136    False
Length: 137, dtype: bool

In [24]:
#RR#Checking Data type
Cleaned_Data.dtypes

IndicatorCode                object
Indicator                    object
Patents_Continent_Code       object
ParentLocation_Continent     object
Country_Code                 object
Country                      object
Gender                       object
Age_Group                    object
Planning_Value_Rate         float64
FactComments                 object
Language_Code                object
Starting_Year                object
Year_Ending                  object
dtype: object

In [25]:
#RR# change data type
Cleaned_Data['Starting_Year'] = pd.to_numeric(Cleaned_Data['Starting_Year'], errors='coerce')
Cleaned_Data['Year_Ending'] = pd.to_numeric(Cleaned_Data['Year_Ending'], errors='coerce')
Cleaned_Data['Gender'] = Cleaned_Data['Gender'].astype('category')

In [26]:
#RR# Cleaned DataFrame
Cleaned_Data.head(10)

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,2021
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN,2020,2021
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN,2020,2021
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,Female,15-49 years,72.1,The global indicator represents all women of r...,EN,2020,2021
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,2020
5,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,EUR,Europe,MDA,Republic of Moldova,Female,15-49 years,59.5,Moldova 2020 Gender and Generations Survey,EN,2020,2020
6,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,ETH,Ethiopia,Female,15-49 years,62.6,The global indicator represents all women of r...,EN,2020,2020
7,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,MDG,Madagascar,Female,15-49 years,65.6,The global indicator represents all women of r...,EN,2020,2020
8,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AMR,Americas,PER,Peru,Female,15-49 years,65.9,The global indicator represents all women of r...,EN,2020,2020
9,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,KEN,Kenya,Female,15-49 years,77.1,Kenya 2020 PMA Survey Phase 2,EN,2020,2020


In [27]:
#RR#change data type
#Create a new column by merging Starting_Year and Year_Ending
# Cleaned_Data['Period_range'] = Cleaned_Data['Starting_Year'].astype(str) + '-' + Cleaned_Data['Year_Ending'].astype(str)
# Cleaned_Data.head(5)

In [28]:
Cleaned_Data.head()

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,2021
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN,2020,2021
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN,2020,2021
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,Female,15-49 years,72.1,The global indicator represents all women of r...,EN,2020,2021
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,2020


In [29]:
# Function to determine the range for each Year_Period
def get_year_range(end_year):
 
    if 2003 <= end_year <= 2005:
        return '2003-2005'
    elif 2006 <= end_year <= 2008:
        return '2006-2008'
    elif 2009 <= end_year <= 2011:
        return '2009-2011'
    elif 2012 <= end_year <= 2014:
        return '2012-2014'
    elif 2015 <= end_year <= 2017:
        return '2015-2017'
    elif 2018 <= end_year <= 2020:
        return '2018-2020'
    elif 2021 <= end_year <= 2023:
        return '2021-2023'
    else:
        return 'Out of Range'

# Apply the function to Year_Period
Cleaned_Data['Period_Range'] = Cleaned_Data['Year_Ending'].apply(get_year_range)

# Display the DataFrame with the new Year_Range column
Cleaned_Data.head(5)

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending,Period_Range
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,2021,2021-2023
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN,2020,2021,2021-2023
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN,2020,2021,2021-2023
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,Female,15-49 years,72.1,The global indicator represents all women of r...,EN,2020,2021,2021-2023
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,2020,2018-2020


In [30]:
#RR# Set the path to the resources folder to exctract CSV of Cleaned Data
# resources_path = "Extracted Cleaned Data File"
# filename = "Cleaned_Data.csv"

# # Save the Cleaned_Data DataFrame to a CSV file in the resources folder
# Cleaned_Data.to_csv(f"{resources_path}/{filename}", index=False)

# print(f"Cleaned_Data successfully exported to {resources_path}/{filename}")

In [31]:
#RR#Checking Data type
Cleaned_Data.dtypes

IndicatorCode                 object
Indicator                     object
Patents_Continent_Code        object
ParentLocation_Continent      object
Country_Code                  object
Country                       object
Gender                      category
Age_Group                     object
Planning_Value_Rate          float64
FactComments                  object
Language_Code                 object
Starting_Year                  int64
Year_Ending                    int64
Period_Range                  object
dtype: object

In [32]:
#HS# Load the data
Hdf = pd.read_csv('Resources/Hdata.csv')

# Display the first few rows of the DataFrame to verify
Hdf.head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,MDG_0000000003,Adolescent birth rate (per 1000 women),text,EUR,Europe,Country,HRV,Croatia,Year,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Registration National Statistics,EN,2024-03-26 00:00:00
1,MDG_0000000003,Adolescent birth rate (per 1000 women),text,EUR,Europe,Country,HRV,Croatia,Year,2023,...,NaN,NaN,NaN,NaN,NaN,6.7,NaN,Registration National Statistics,EN,2024-03-26 00:00:00
2,MDG_0000000003,Adolescent birth rate (per 1000 women),text,EMR,Eastern Mediterranean,Country,YEM,Yemen,Year,2023,...,NaN,NaN,NaN,NaN,NaN,77.0,NaN,Yemen 2022-2023 Multiple Indicator Cluster Survey,EN,2024-03-26 00:00:00
3,MDG_0000000003,Adolescent birth rate (per 1000 women),text,GLOBAL,Global,Country,MAC,"China, Macao Special Administrative Region",Year,2022,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Registration National Statistics & WPP2022,EN,2024-03-26 00:00:00
4,MDG_0000000003,Adolescent birth rate (per 1000 women),text,EUR,Europe,Country,DNK,Denmark,Year,2022,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Registration National Statistics & WPP2022,EN,2024-03-26 00:00:00


In [33]:
Hdf_cleaned = Hdf[['ParentLocation', 'Location', 'Period', 'Dim2', 'Value', 'FactComments']]

Hdf_cleaned.head()

,ParentLocation,Location,Period,Dim2,Value,FactComments
0,Europe,Croatia,2023,10-14 years,0.0,Registration National Statistics
1,Europe,Croatia,2023,15-19 years,6.7,Registration National Statistics
2,Eastern Mediterranean,Yemen,2023,15-19 years,77.0,Yemen 2022-2023 Multiple Indicator Cluster Survey
3,Global,"China, Macao Special Administrative Region",2022,10-14 years,0.0,Registration National Statistics & WPP2022
4,Europe,Denmark,2022,10-14 years,0.0,Registration National Statistics & WPP2022


In [34]:
#HS# Create a copy of the DataFrame
hDf_cleaned = Hdf_cleaned.copy()

# Rename the 'Value' column to 'Adolescent birth rate (per 1000 women)'
hDf_cleaned.rename(columns={'Value': 'Adolescent_birth_rate', 'Dim2': 'Age-group'}, inplace=True)
hDf_cleaned.head()

,ParentLocation,Location,Period,Age-group,Adolescent_birth_rate,FactComments
0,Europe,Croatia,2023,10-14 years,0.0,Registration National Statistics
1,Europe,Croatia,2023,15-19 years,6.7,Registration National Statistics
2,Eastern Mediterranean,Yemen,2023,15-19 years,77.0,Yemen 2022-2023 Multiple Indicator Cluster Survey
3,Global,"China, Macao Special Administrative Region",2022,10-14 years,0.0,Registration National Statistics & WPP2022
4,Europe,Denmark,2022,10-14 years,0.0,Registration National Statistics & WPP2022


In [35]:
# Export the cleaned data to CSV
#hDf_cleaned.to_csv("Output/Halima.csv", index=False)

In [36]:
#HS# Group by 'Location' (country) and find the maximum Adolescent birth rate
max_birth_rate =hDf_cleaned.groupby('Location')['Adolescent_birth_rate'].max()

# Find the country with the highest Adolescent birth rate
highest_birth_rate_country = max_birth_rate.idxmax()
highest_birth_rate_value = max_birth_rate.max()

print(f"The country with the highest Adolescent birth rate (per 1000 women) is {highest_birth_rate_country} with a rate of {highest_birth_rate_value}")

The country with the highest Adolescent birth rate (per 1000 women) is Central African Republic with a rate of 184.4


In [37]:
min_birth_rate = hDf_cleaned.groupby('Location')['Adolescent_birth_rate'].min()

#HS# Find the country with the lowest Adolescent birth rate
lowest_birth_rate_country = min_birth_rate.idxmin()
lowest_birth_rate_value = min_birth_rate.min()

print(f"The country with the lowest Adolescent birth rate (per 1000 women) is {lowest_birth_rate_country} with a rate of {lowest_birth_rate_value}")

The country with the lowest Adolescent birth rate (per 1000 women) is Algeria with a rate of 0.0


In [38]:
highest_rate = hDf_cleaned.sort_values(by='Adolescent_birth_rate', ascending=False)

highest_rate.head(10)

,ParentLocation,Location,Period,Age-group,Adolescent_birth_rate,FactComments
310,Africa,Central African Republic,2018,15-19 years,184.4,Central African Republic 2018-2019 Multiple In...
409,Africa,Equatorial Guinea,2010,15-19 years,176.0,Equatorial Guinea 2011 Demographic and Health ...
386,Africa,Angola,2014,15-19 years,162.7,Angola 2015-2016 Demographic and Health Survey
418,Africa,South Sudan,2008,15-19 years,158.1,South Sudan 2010 Household Health Survey Secon...
149,Africa,Mozambique,2021,15-19 years,158.0,Mozambique 2022-2023 Demographic and Health Su...
233,Africa,Niger,2020,15-19 years,150.3,Niger 2021 Malaria Indicator Survey
231,Africa,Mali,2020,15-19 years,144.8,Mali 2021 Malaria Indicators Survey
267,Africa,Madagascar,2019,15-19 years,143.0,Madagascar 2021 Demographic and Health Survey
307,Africa,Chad,2018,15-19 years,138.5,Chad 2019 Multiple Indicator Cluster Survey
306,Africa,Malawi,2018,15-19 years,135.6,Malawi 2019-2020 Multiple Indicator Cluster Su...


In [39]:
#HS# Sort the DataFrame in ascending order based on 'Adolescent birth rate (per 1000 women)'
lowest_rate = hDf_cleaned.sort_values(by='Adolescent_birth_rate', ascending=True)

# Display the sorted DataFrame
lowest_rate.head(10)

,ParentLocation,Location,Period,Age-group,Adolescent_birth_rate,FactComments
0,Europe,Croatia,2023,10-14 years,0.0,Registration National Statistics
74,South-East Asia,Bhutan,2021,10-14 years,0.0,Registration National Statistics & WPP2022
73,Americas,Bermuda,2021,10-14 years,0.0,Registration National Statistics & WPP2022
72,Europe,Belgium,2021,10-14 years,0.0,Registration Eurostat
71,Europe,Azerbaijan,2021,10-14 years,0.0,Registration National Statistics & WPP2022
70,Europe,Austria,2021,10-14 years,0.0,Registration Eurostat
211,Europe,Armenia,2020,10-14 years,0.0,Registration Eurostat
212,Americas,Montserrat,2020,10-14 years,0.0,Registration National Statistics & WPP2022
213,South-East Asia,Nepal,2020,10-14 years,0.0,Nepal 2022 Demographic and Health Survey
254,Eastern Mediterranean,Bahrain,2019,10-14 years,0.0,Registration National Statistics & WPP2022


In [40]:
#HS# Count occurrences of Age-group values
age_group_counts = hDf_cleaned['Age-group'].value_counts()

# Display the counts
print("Age-group counts:")
print(age_group_counts)

Age-group counts:
Age-group
15-19 years    223
10-14 years    215
Name: count, dtype: int64


In [41]:
# Adolescent data frame
hDf_cleaned.dtypes

ParentLocation            object
Location                  object
Period                     int64
Age-group                 object
Adolescent_birth_rate    float64
FactComments              object
dtype: object

In [42]:
## HS
# Add Starting_Year and Year_Ending columns
hDf_cleaned['Starting_Year'] = hDf_cleaned['Period']
hDf_cleaned['Year_Ending'] = hDf_cleaned['Period']

In [43]:
##HS
#  Define the function to categorize Period into ranges
def categorize_period(period):
    if 2003 <= period <= 2005:
        return '2003-2005'
    elif 2006 <= period <= 2008:
        return '2006-2008'
    elif 2009 <= period <= 2011:
        return '2009-2011'
    elif 2012 <= period <= 2014:
        return '2012-2014'
    elif 2015 <= period <= 2017:
        return '2015-2017'
    elif 2018 <= period <= 2020:
        return '2018-2020'
    elif 2021 <= period <= 2023:
        return '2021-2023'
    else:
        return 'Other'  # If the period does not fall into these ranges

# Add Period_Range column
hDf_cleaned['Period_Range'] = hDf_cleaned['Period'].apply(categorize_period)

In [44]:
hDf_cleaned = hDf_cleaned.drop(["Period"], axis=1)
hDf_cleaned.head()

,ParentLocation,Location,Age-group,Adolescent_birth_rate,FactComments,Starting_Year,Year_Ending,Period_Range
0,Europe,Croatia,10-14 years,0.0,Registration National Statistics,2023,2023,2021-2023
1,Europe,Croatia,15-19 years,6.7,Registration National Statistics,2023,2023,2021-2023
2,Eastern Mediterranean,Yemen,15-19 years,77.0,Yemen 2022-2023 Multiple Indicator Cluster Survey,2023,2023,2021-2023
3,Global,"China, Macao Special Administrative Region",10-14 years,0.0,Registration National Statistics & WPP2022,2022,2022,2021-2023
4,Europe,Denmark,10-14 years,0.0,Registration National Statistics & WPP2022,2022,2022,2021-2023


In [45]:
hDf_cleaned.dtypes

ParentLocation            object
Location                  object
Age-group                 object
Adolescent_birth_rate    float64
FactComments              object
Starting_Year              int64
Year_Ending                int64
Period_Range              object
dtype: object

In [46]:
#LR#
csvpath = Path('.', 'Resources', 'births-attended-by-skilled-health-personnel.csv')

In [47]:
#LR# Read the CSV into a Pandas DataFrame
skilled_personnel_df = pd.read_csv(csvpath, low_memory=False)

# Print the first rows of data to the screen
skilled_personnel_df.head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,MDG_0000000025,Births attended by skilled health personnel (%),text,SEAR,South-East Asia,Country,IDN,Indonesia,Year,2023,...,NaN,NaN,NaN,NaN,NaN,95.7,NaN,SUSENAS 2023 as part of Welfare Statistics 2023,EN,2024-04-26 00:00:00
1,MDG_0000000025,Births attended by skilled health personnel (%),text,EUR,Europe,Country,FRA,France,Year,2023,...,NaN,NaN,NaN,NaN,NaN,97.6,NaN,Institutional Birth (IB): includes all deliver...,EN,2024-04-26 00:00:00
2,MDG_0000000025,Births attended by skilled health personnel (%),text,EMR,Eastern Mediterranean,Country,SAU,Saudi Arabia,Year,2023,...,NaN,NaN,NaN,NaN,NaN,99.8,NaN,Women Health and Reproductive Survey,EN,2024-04-26 00:00:00
3,MDG_0000000025,Births attended by skilled health personnel (%),text,EMR,Eastern Mediterranean,Country,JOR,Jordan,Year,2023,...,NaN,NaN,NaN,NaN,NaN,99.9,NaN,Department of Statistics (DoS) [Jordan] and IC...,EN,2024-04-26 00:00:00
4,MDG_0000000025,Births attended by skilled health personnel (%),text,EMR,Eastern Mediterranean,Country,YEM,Yemen,Year,2022-2023,...,NaN,NaN,NaN,NaN,NaN,60.9,NaN,Central Statistical Organization (CSO) and Uni...,EN,2024-04-26 00:00:00


In [48]:
#LR#make a copy of dataframe and drop unwanted columns
clean_df_skilled_personnel = skilled_personnel_df.drop(['Indicator','ValueType', 'FactValueUoM', 'Location type','FactValueNumericLowPrefix', 'IsLatestYear', 'FactValueNumeric',
                                                        'FactValueNumericPrefix','FactValueNumericLow','FactValueTranslationID', 'Dim2', 'Dim2 type', 'Dim1 type', 'Language',
                                                        'Dim2ValueCode', 'Dim3', 'DataSourceDimValueCode', 'Dim3ValueCode', 'DataSource', 'Dim1ValueCode', 'Dim3 type',
                                                        'Dim1', 'FactValueNumericHighPrefix', 'FactValueNumericHigh'], axis=1)
clean_df_skilled_personnel.head()

,IndicatorCode,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period type,Period,Value,FactComments,DateModified
0,MDG_0000000025,SEAR,South-East Asia,IDN,Indonesia,Year,2023,95.7,SUSENAS 2023 as part of Welfare Statistics 2023,2024-04-26 00:00:00
1,MDG_0000000025,EUR,Europe,FRA,France,Year,2023,97.6,Institutional Birth (IB): includes all deliver...,2024-04-26 00:00:00
2,MDG_0000000025,EMR,Eastern Mediterranean,SAU,Saudi Arabia,Year,2023,99.8,Women Health and Reproductive Survey,2024-04-26 00:00:00
3,MDG_0000000025,EMR,Eastern Mediterranean,JOR,Jordan,Year,2023,99.9,Department of Statistics (DoS) [Jordan] and IC...,2024-04-26 00:00:00
4,MDG_0000000025,EMR,Eastern Mediterranean,YEM,Yemen,Year,2022-2023,60.9,Central Statistical Organization (CSO) and Uni...,2024-04-26 00:00:00


In [49]:
clean_df_skilled_personnel.rename(columns={'Value': 'skilled_personnel%'}, inplace=True)

In [50]:
#LR# print data types
clean_df_skilled_personnel.dtypes

IndicatorCode           object
ParentLocationCode      object
ParentLocation          object
SpatialDimValueCode     object
Location                object
Period type             object
Period                  object
skilled_personnel%     float64
FactComments            object
DateModified            object
dtype: object

In [51]:
#LR# Split  years by pattern '-'
clean_df_skilled_personnel['Period'].str.split(pat='-')

0             [2023]
1             [2023]
2             [2023]
3             [2023]
4       [2022, 2023]
            ...     
2337    [1999, 2000]
2338    [1999, 2000]
2339    [1999, 2000]
2340    [1999, 2000]
2341    [1999, 2000]
Name: Period, Length: 2342, dtype: object

In [52]:
#LR# checking plit where 0=starting year=LHS values
clean_df_skilled_personnel['Period'].str.split(pat='-').str[0]

0       2023
1       2023
2       2023
3       2023
4       2022
        ... 
2337    1999
2338    1999
2339    1999
2340    1999
2341    1999
Name: Period, Length: 2342, dtype: object

In [53]:
#LR# adding left hand values to new column name 'Starting_Year'
clean_df_skilled_personnel['Starting_Year'] = clean_df_skilled_personnel['Period'].str.split(pat='-').str[0]

In [54]:
#LR# checking plit where 1=year_Ending=RHS values
clean_df_skilled_personnel['Period'].str.split(pat='-').str[1]

0        NaN
1        NaN
2        NaN
3        NaN
4       2023
        ... 
2337    2000
2338    2000
2339    2000
2340    2000
2341    2000
Name: Period, Length: 2342, dtype: object

In [55]:
#LR# adding Right hand values to new column name 'Year_Ending'
clean_df_skilled_personnel['Year_Ending'] = clean_df_skilled_personnel['Period'].str.split(pat='-').str[1]
# checking null values for column 'year_Ending'
clean_df_skilled_personnel.isnull()

,IndicatorCode,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period type,Period,skilled_personnel%,FactComments,DateModified,Starting_Year,Year_Ending
0,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2337,False,False,False,False,False,False,False,False,False,False,False,False
2338,False,False,False,False,False,False,False,False,False,False,False,False
2339,False,False,False,False,False,False,False,False,False,False,False,False
2340,False,False,False,False,False,False,False,False,False,False,False,False


In [56]:
#LR# checking records who is carrying null values for year_ending column
clean_df_skilled_personnel[clean_df_skilled_personnel['Year_Ending'].isna()==1]

,IndicatorCode,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period type,Period,skilled_personnel%,FactComments,DateModified,Starting_Year,Year_Ending
0,MDG_0000000025,SEAR,South-East Asia,IDN,Indonesia,Year,2023,95.7,SUSENAS 2023 as part of Welfare Statistics 2023,2024-04-26 00:00:00,2023,NaN
1,MDG_0000000025,EUR,Europe,FRA,France,Year,2023,97.6,Institutional Birth (IB): includes all deliver...,2024-04-26 00:00:00,2023,NaN
2,MDG_0000000025,EMR,Eastern Mediterranean,SAU,Saudi Arabia,Year,2023,99.8,Women Health and Reproductive Survey,2024-04-26 00:00:00,2023,NaN
3,MDG_0000000025,EMR,Eastern Mediterranean,JOR,Jordan,Year,2023,99.9,Department of Statistics (DoS) [Jordan] and IC...,2024-04-26 00:00:00,2023,NaN
9,MDG_0000000025,EUR,Europe,ALB,Albania,Year,2022,100.0,Unclear definition of the cadres included. Ins...,2024-04-26 00:00:00,2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2331,MDG_0000000025,EUR,Europe,FIN,Finland,Year,2000,99.9,Unclear definition of the cadres included. Sta...,2024-04-26 00:00:00,2000,NaN
2332,MDG_0000000025,EUR,Europe,GRC,Greece,Year,2000,99.9,Unclear definition of the cadres included. Hel...,2024-04-26 00:00:00,2000,NaN
2333,MDG_0000000025,WPR,Western Pacific,KOR,Republic of Korea,Year,2000,99.9,Institutional Birth (IB): includes all deliver...,2024-04-26 00:00:00,2000,NaN
2334,MDG_0000000025,EUR,Europe,SVN,Slovenia,Year,2000,99.9,Statistical Yearbook 2001. Ljubljana: Statisti...,2024-04-26 00:00:00,2000,NaN


In [57]:
#LR# replacing null values by zero
#Cleaned_Data['Year_Ending'] = Cleaned_Data['Year_Ending'].fillna(0)
clean_df_skilled_personnel['Year_Ending'] = clean_df_skilled_personnel['Year_Ending'].fillna(clean_df_skilled_personnel['Starting_Year'])
clean_df_skilled_personnel.head(10)

,IndicatorCode,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period type,Period,skilled_personnel%,FactComments,DateModified,Starting_Year,Year_Ending
0,MDG_0000000025,SEAR,South-East Asia,IDN,Indonesia,Year,2023,95.7,SUSENAS 2023 as part of Welfare Statistics 2023,2024-04-26 00:00:00,2023,2023
1,MDG_0000000025,EUR,Europe,FRA,France,Year,2023,97.6,Institutional Birth (IB): includes all deliver...,2024-04-26 00:00:00,2023,2023
2,MDG_0000000025,EMR,Eastern Mediterranean,SAU,Saudi Arabia,Year,2023,99.8,Women Health and Reproductive Survey,2024-04-26 00:00:00,2023,2023
3,MDG_0000000025,EMR,Eastern Mediterranean,JOR,Jordan,Year,2023,99.9,Department of Statistics (DoS) [Jordan] and IC...,2024-04-26 00:00:00,2023,2023
4,MDG_0000000025,EMR,Eastern Mediterranean,YEM,Yemen,Year,2022-2023,60.9,Central Statistical Organization (CSO) and Uni...,2024-04-26 00:00:00,2022,2023
5,MDG_0000000025,EMR,Eastern Mediterranean,AFG,Afghanistan,Year,2022-2023,67.5,UNICEF. 2023. Afghanistan Multiple Indicator C...,2024-04-26 00:00:00,2022,2023
6,MDG_0000000025,AFR,Africa,MOZ,Mozambique,Year,2022-2023,67.5,Instituto Nacional de Estatística (INE) e ICF....,2024-04-26 00:00:00,2022,2023
7,MDG_0000000025,AFR,Africa,GHA,Ghana,Year,2022-2023,87.6,Ghana Statistical Service (GSS) and ICF. 2023....,2024-04-26 00:00:00,2022,2023
8,MDG_0000000025,EMR,Eastern Mediterranean,IRN,Iran (Islamic Republic of),Year,2022-2023,99.0,Health Indicators in the Islamic Republic of I...,2024-04-26 00:00:00,2022,2023
9,MDG_0000000025,EUR,Europe,ALB,Albania,Year,2022,100.0,Unclear definition of the cadres included. Ins...,2024-04-26 00:00:00,2022,2022


In [58]:
#LR# 
clean_df_skilled_personnel=clean_df_skilled_personnel.drop(['Period'],axis=1)
clean_df_skilled_personnel.head(5)

,IndicatorCode,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period type,skilled_personnel%,FactComments,DateModified,Starting_Year,Year_Ending
0,MDG_0000000025,SEAR,South-East Asia,IDN,Indonesia,Year,95.7,SUSENAS 2023 as part of Welfare Statistics 2023,2024-04-26 00:00:00,2023,2023
1,MDG_0000000025,EUR,Europe,FRA,France,Year,97.6,Institutional Birth (IB): includes all deliver...,2024-04-26 00:00:00,2023,2023
2,MDG_0000000025,EMR,Eastern Mediterranean,SAU,Saudi Arabia,Year,99.8,Women Health and Reproductive Survey,2024-04-26 00:00:00,2023,2023
3,MDG_0000000025,EMR,Eastern Mediterranean,JOR,Jordan,Year,99.9,Department of Statistics (DoS) [Jordan] and IC...,2024-04-26 00:00:00,2023,2023
4,MDG_0000000025,EMR,Eastern Mediterranean,YEM,Yemen,Year,60.9,Central Statistical Organization (CSO) and Uni...,2024-04-26 00:00:00,2022,2023


In [59]:
#LR# change data type
clean_df_skilled_personnel['Starting_Year'] = pd.to_numeric(clean_df_skilled_personnel['Starting_Year'], errors='coerce')
clean_df_skilled_personnel['Year_Ending'] = pd.to_numeric(clean_df_skilled_personnel['Year_Ending'], errors='coerce')

In [60]:
#LR#  Function to determine the range for each Year_Period
def get_year_range(end_year):
 
    if 2003 <= end_year <= 2005:
        return '2003-2005'
    elif 2006 <= end_year <= 2008:
        return '2006-2008'
    elif 2009 <= end_year <= 2011:
        return '2009-2011'
    elif 2012 <= end_year <= 2014:
        return '2012-2014'
    elif 2015 <= end_year <= 2017:
        return '2015-2017'
    elif 2018 <= end_year <= 2020:
        return '2018-2020'
    elif 2021 <= end_year <= 2023:
        return '2021-2023'
    else:
        return 'Out of Range'

# Apply the function to Year_Period
clean_df_skilled_personnel['Period_Range'] = clean_df_skilled_personnel['Year_Ending'].apply(get_year_range)

# Display the DataFrame with the new Year_Range column
Cleaned_Data.head(5)

,IndicatorCode,Indicator,Patents_Continent_Code,ParentLocation_Continent,Country_Code,Country,Gender,Age_Group,Planning_Value_Rate,FactComments,Language_Code,Starting_Year,Year_Ending,Period_Range
0,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,UGA,Uganda,Female,15-49 years,65.5,The global indicator represents all women of r...,EN,2021,2021,2021-2023
1,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,NER,Niger,Female,15-49 years,36.9,Niger 2021 PMA Survey Phase 1,EN,2020,2021,2021-2023
2,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,BFA,Burkina Faso,Female,15-49 years,58.2,Burkina Faso 2021 PMA Survey Phase 2,EN,2020,2021,2021-2023
3,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,WPR,Western Pacific,VNM,Viet Nam,Female,15-49 years,72.1,The global indicator represents all women of r...,EN,2020,2021,2021-2023
4,SDGFPALL,Women of reproductive age (aged 15-49 years) w...,AFR,Africa,CIV,Cote d'Ivoire,Female,15-49 years,46.1,Côte d'Ivoire 2020 PMA Survey Phase 1,EN,2020,2020,2018-2020


In [67]:
clean_df_skilled_personnel.head()

,IndicatorCode,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period type,skilled_personnel%,FactComments,DateModified,Starting_Year,Year_Ending,Period_Range
0,MDG_0000000025,SEAR,South-East Asia,IDN,Indonesia,Year,95.7,SUSENAS 2023 as part of Welfare Statistics 2023,2024-04-26 00:00:00,2023,2023,2021-2023
1,MDG_0000000025,EUR,Europe,FRA,France,Year,97.6,Institutional Birth (IB): includes all deliver...,2024-04-26 00:00:00,2023,2023,2021-2023
2,MDG_0000000025,EMR,Eastern Mediterranean,SAU,Saudi Arabia,Year,99.8,Women Health and Reproductive Survey,2024-04-26 00:00:00,2023,2023,2021-2023
3,MDG_0000000025,EMR,Eastern Mediterranean,JOR,Jordan,Year,99.9,Department of Statistics (DoS) [Jordan] and IC...,2024-04-26 00:00:00,2023,2023,2021-2023
4,MDG_0000000025,EMR,Eastern Mediterranean,YEM,Yemen,Year,60.9,Central Statistical Organization (CSO) and Uni...,2024-04-26 00:00:00,2022,2023,2021-2023


In [68]:
#LR#
parent_location = clean_df_skilled_personnel.groupby(["ParentLocation"])["skilled_personnel%"].value_counts()
pd.DataFrame(parent_location)

count
ParentLocation  skilled_personnel%       
Africa          99.8                   12
                99.0                    8
                99.7                    8
                99.5                    6
                43.7                    3
...                                   ...
Western Pacific 97.5                    1
                97.6                    1
                97.8                    1
                98.2                    1
                98.9                    1

[745 rows x 1 columns]

In [71]:
#LR# remove year <= 2002
clean_df_skilled_personnel = clean_df_skilled_personnel.loc[(clean_df_skilled_personnel["Period_Range"]<= 2002)]
clean_df_skilled_personnel.head()

TypeError: '<=' not supported between instances of 'str' and 'int'

In [73]:
#IA# Read the CSV into a Pandas DataFrame
csvpath = Path('.', 'Resources', 'antenatal-care-coverage.csv')
#IW# Read the CSV into a Pandas DataFrame
care_coverage_df = pd.read_csv(csvpath, low_memory=False)

# Print the first rows of data to the screen
care_coverage_df.head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,WHS4_154,Antenatal care coverage - at least four visits...,text,EMR,Eastern Mediterranean,Country,SAU,Saudi Arabia,Year,2023,...,NaN,NaN,NaN,NaN,NaN,88.8,NaN,Women Health and Reproductive Care Survey 2023.,EN,2024-05-13 00:00:00
1,WHS4_154,Antenatal care coverage - at least four visits...,text,AFR,Africa,Country,SEN,Senegal,Year,2021-2023,...,NaN,NaN,NaN,NaN,NaN,68.4,NaN,DHS 2023 (DHS Continuous) Key indicators.,EN,2024-05-13 00:00:00
2,WHS4_154,Antenatal care coverage - at least four visits...,text,EMR,Eastern Mediterranean,Country,TUN,Tunisia,Year,2021-2023,...,NaN,NaN,NaN,NaN,NaN,79,NaN,MICS 2023 (snaphots),EN,2024-05-13 00:00:00
3,WHS4_154,Antenatal care coverage - at least four visits...,text,EMR,Eastern Mediterranean,Country,JOR,Jordan,Year,2021-2023,...,NaN,NaN,NaN,NaN,NaN,93.1,NaN,PFHS 2023 (key indicators).,EN,2024-05-13 00:00:00
4,WHS4_154,Antenatal care coverage - at least four visits...,text,EMR,Eastern Mediterranean,Country,YEM,Yemen,Year,2020-2023,...,NaN,NaN,NaN,NaN,NaN,32.4,NaN,MICS 2022-2023.,EN,2024-05-13 00:00:00


In [74]:
#IA# 
cleaned_care_coverage_df = care_coverage_df[['IndicatorCode', 'Indicator', 'ParentLocation', 'Location', 'Period', 'Value', 'FactComments']]
cleaned_care_coverage_df.head()

,IndicatorCode,Indicator,ParentLocation,Location,Period,Value,FactComments
0,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Saudi Arabia,2023,88.8,Women Health and Reproductive Care Survey 2023.
1,WHS4_154,Antenatal care coverage - at least four visits...,Africa,Senegal,2021-2023,68.4,DHS 2023 (DHS Continuous) Key indicators.
2,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Tunisia,2021-2023,79,MICS 2023 (snaphots)
3,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Jordan,2021-2023,93.1,PFHS 2023 (key indicators).
4,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Yemen,2020-2023,32.4,MICS 2022-2023.


In [75]:
#IA# Create a copy of the DataFrame
cleaned_care_coverage_df = cleaned_care_coverage_df.copy()
# Rename the 'Value' column to 'Adolescent birth rate (per 1000 women)'

cleaned_care_coverage_df.rename(columns={'Value': 'Anatal_care%'}, inplace=True)

In [82]:
#IA# Split  years by pattern '-'
cleaned_care_coverage_df['Period'].str.split(pat='-')

0             [2023]
1       [2021, 2023]
2       [2021, 2023]
3       [2021, 2023]
4       [2020, 2023]
            ...     
1096    [1985, 1991]
1097    [1985, 1990]
1098    [1985, 1990]
1099    [1985, 1990]
1100    [1985, 1990]
Name: Period, Length: 1101, dtype: object

In [83]:
#IA# checking plit where 0=starting year=LHS values
cleaned_care_coverage_df['Period'].str.split(pat='-').str[0]

0       2023
1       2021
2       2021
3       2021
4       2020
        ... 
1096    1985
1097    1985
1098    1985
1099    1985
1100    1985
Name: Period, Length: 1101, dtype: object

In [84]:
#IA# adding left hand values to new column name 'Starting_Year'
cleaned_care_coverage_df['Starting_Year'] = cleaned_care_coverage_df['Period'].str.split(pat='-').str[0]

In [86]:
#IA# checking plit where 1=year_Ending=RHS values
cleaned_care_coverage_df['Period'].str.split(pat='-').str[1]

0        NaN
1       2023
2       2023
3       2023
4       2023
        ... 
1096    1991
1097    1990
1098    1990
1099    1990
1100    1990
Name: Period, Length: 1101, dtype: object

In [87]:
#IA# adding Right hand values to new column name 'Year_Ending'
cleaned_care_coverage_df['Year_Ending'] = cleaned_care_coverage_df['Period'].str.split(pat='-').str[1]
# checking null values for column 'year_Ending'
cleaned_care_coverage_df.isnull()

,IndicatorCode,Indicator,ParentLocation,Location,Period,Anatal_care%,FactComments,Starting_Year,Year_Ending
0,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
1096,False,False,False,False,False,False,False,False,False
1097,False,False,False,False,False,False,False,False,False
1098,False,False,False,False,False,False,False,False,False
1099,False,False,False,False,False,False,False,False,False


In [88]:
#IA# checking records who is carrying null values for year_ending column
cleaned_care_coverage_df[cleaned_care_coverage_df['Year_Ending'].isna()==1]

,IndicatorCode,Indicator,ParentLocation,Location,Period,Anatal_care%,FactComments,Starting_Year,Year_Ending
0,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Saudi Arabia,2023,88.8,Women Health and Reproductive Care Survey 2023.,2023,NaN
7,WHS4_154,Antenatal care coverage - at least four visits...,Africa,Burundi,2022,34.9,Annuaire des Statistisques Sanitaires 2022.,2022,NaN
8,WHS4_154,Antenatal care coverage - at least four visits...,Americas,Antigua and Barbuda,2022,75,PLISA Health Information Platform; 2024.,2022,NaN
9,WHS4_154,Antenatal care coverage - at least four visits...,Americas,Bolivia (Plurinational State of),2022,81.3,PLISA Health Information Platform; 2024.,2022,NaN
10,WHS4_154,Antenatal care coverage - at least four visits...,Americas,Saint Kitts and Nevis,2022,85,PLISA Health Information Platform; 2024.,2022,NaN
...,...,...,...,...,...,...,...,...,...
1060,WHS4_154,Antenatal care coverage - at least four visits...,Europe,Estonia,1994,92.1,National Institute for Health Development [Est...,1994,NaN
1066,WHS4_154,Antenatal care coverage - at least four visits...,Europe,Estonia,1993,91.9,National Institute for Health Development [Est...,1993,NaN
1067,WHS4_154,Antenatal care coverage - at least four visits...,Europe,Czechia,1993,96.8,6 or more visits. RHS 1993.,1993,NaN
1077,WHS4_154,Antenatal care coverage - at least four visits...,Europe,Estonia,1992,92.1,National Institute for Health Development [Est...,1992,NaN


In [89]:
#IA# replacing null values by zero
#Cleaned_Data['Year_Ending'] = Cleaned_Data['Year_Ending'].fillna(0)
cleaned_care_coverage_df['Year_Ending'] = cleaned_care_coverage_df['Year_Ending'].fillna(cleaned_care_coverage_df['Starting_Year'])
cleaned_care_coverage_df.head(10)

,IndicatorCode,Indicator,ParentLocation,Location,Period,Anatal_care%,FactComments,Starting_Year,Year_Ending
0,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Saudi Arabia,2023,88.8,Women Health and Reproductive Care Survey 2023.,2023,2023
1,WHS4_154,Antenatal care coverage - at least four visits...,Africa,Senegal,2021-2023,68.4,DHS 2023 (DHS Continuous) Key indicators.,2021,2023
2,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Tunisia,2021-2023,79,MICS 2023 (snaphots),2021,2023
3,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Jordan,2021-2023,93.1,PFHS 2023 (key indicators).,2021,2023
4,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Yemen,2020-2023,32.4,MICS 2022-2023.,2020,2023
5,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Afghanistan,2020-2023,33.4,MICS 2022-2023.,2020,2023
6,WHS4_154,Antenatal care coverage - at least four visits...,Africa,Mozambique,2020-2023,48.6,DHS 2022-2023. Key indicators.,2020,2023
7,WHS4_154,Antenatal care coverage - at least four visits...,Africa,Burundi,2022,34.9,Annuaire des Statistisques Sanitaires 2022.,2022,2022
8,WHS4_154,Antenatal care coverage - at least four visits...,Americas,Antigua and Barbuda,2022,75,PLISA Health Information Platform; 2024.,2022,2022
9,WHS4_154,Antenatal care coverage - at least four visits...,Americas,Bolivia (Plurinational State of),2022,81.3,PLISA Health Information Platform; 2024.,2022,2022


In [90]:
#IA# 
cleaned_care_coverage_df=cleaned_care_coverage_df.drop(['Period'],axis=1)
cleaned_care_coverage_df.head(5)

,IndicatorCode,Indicator,ParentLocation,Location,Anatal_care%,FactComments,Starting_Year,Year_Ending
0,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Saudi Arabia,88.8,Women Health and Reproductive Care Survey 2023.,2023,2023
1,WHS4_154,Antenatal care coverage - at least four visits...,Africa,Senegal,68.4,DHS 2023 (DHS Continuous) Key indicators.,2021,2023
2,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Tunisia,79,MICS 2023 (snaphots),2021,2023
3,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Jordan,93.1,PFHS 2023 (key indicators).,2021,2023
4,WHS4_154,Antenatal care coverage - at least four visits...,Eastern Mediterranean,Yemen,32.4,MICS 2022-2023.,2020,2023


In [93]:
#IA# change data type
cleaned_care_coverage_df['Starting_Year'] = pd.to_numeric(cleaned_care_coverage_df['Starting_Year'], errors='coerce')
cleaned_care_coverage_df['Year_Ending'] = pd.to_numeric(cleaned_care_coverage_df['Year_Ending'], errors='coerce')

In [94]:
##IA
#  Define the function to categorize Period into ranges
def categorize_period(period):
    if 2003 <= period <= 2005:
        return '2003-2005'
    elif 2006 <= period <= 2008:
        return '2006-2008'
    elif 2009 <= period <= 2011:
        return '2009-2011'
    elif 2012 <= period <= 2014:
        return '2012-2014'
    elif 2015 <= period <= 2017:
        return '2015-2017'
    elif 2018 <= period <= 2020:
        return '2018-2020'
    elif 2021 <= period <= 2023:
        return '2021-2023'
    else:
        return 'Other'  # If the period does not fall into these ranges

# Add Period_Range column
cleaned_care_coverage_df['Period_Range'] = cleaned_care_coverage_df['Year_Ending'].apply(categorize_period)

In [95]:
clean_df_skilled_personnel.head()

,IndicatorCode,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period type,skilled_personnel%,FactComments,DateModified,Starting_Year,Year_Ending,Period_Range
0,MDG_0000000025,SEAR,South-East Asia,IDN,Indonesia,Year,95.7,SUSENAS 2023 as part of Welfare Statistics 2023,2024-04-26 00:00:00,2023,2023,2021-2023
1,MDG_0000000025,EUR,Europe,FRA,France,Year,97.6,Institutional Birth (IB): includes all deliver...,2024-04-26 00:00:00,2023,2023,2021-2023
2,MDG_0000000025,EMR,Eastern Mediterranean,SAU,Saudi Arabia,Year,99.8,Women Health and Reproductive Survey,2024-04-26 00:00:00,2023,2023,2021-2023
3,MDG_0000000025,EMR,Eastern Mediterranean,JOR,Jordan,Year,99.9,Department of Statistics (DoS) [Jordan] and IC...,2024-04-26 00:00:00,2023,2023,2021-2023
4,MDG_0000000025,EMR,Eastern Mediterranean,YEM,Yemen,Year,60.9,Central Statistical Organization (CSO) and Uni...,2024-04-26 00:00:00,2022,2023,2021-2023


In [96]:
#IA#
cleaned_care_coverage_df.dtypes

IndicatorCode     object
Indicator         object
ParentLocation    object
Location          object
Anatal_care%      object
FactComments      object
Starting_Year      int64
Year_Ending        int64
Period_Range      object
dtype: object